# Data engineering capstone - Eleflow

#### Instalando dependências

In [33]:
!pip install pandas requests pendulum pandasql

#### imports necessários

In [108]:
import os
import json
import pendulum
import requests
import pandas as pd
from pandasql import sqldf
import sqlite3

## Carregar os dados de VRA
- Normalizar o cabeçalho para snake case
- Salvar estes dados

In [3]:
# Função para percorrer o diretório, pegar todos os arquivos e carregar as informações
def ler_jsons(diretorio):
    arquivos_json = [arquivo for arquivo in os.listdir(diretorio) if arquivo.endswith('.json')]

    for nome_arquivo in arquivos_json:
        caminho_arquivo = os.path.join(diretorio, nome_arquivo)
        with open(caminho_arquivo, 'r', encoding='utf-8-sig') as arquivo:
            yield json.load(arquivo)

In [4]:
# Nome das colunas em snake_case
nomes_colunas_vra = ['icao_empresa_aerea', 'numero_voo', 'codigo_autorizacao', 'codigo_tipo_linha',
                     'icao_aerodromo_origem', 'icao_aerodromo_destino', 'partida_prevista',
                     'partida_real', 'chegada_prevista', 'chegada_real', 'situacao_voo', 'codigo_justificativa']


In [5]:
# Diretório onde estão os arquivos JSON
diretorio_vra = './data_engineer_test/VRA'

In [7]:
# Criar um gerador
vra_generator = ler_jsons(diretorio_vra)

In [8]:
# Criação do dataframe com os dados de vra
vra_df = pd.DataFrame()

In [9]:
# Adicionando informações de cada arquivo JSON no dataframe
for lista_de_dicts in vra_generator:
    vra_df = vra_df._append(lista_de_dicts, ignore_index=True)

In [10]:
# Modificando nome das colunas do dataframe VRA
vra_df.columns = nomes_colunas_vra

In [11]:
print(vra_df)

       icao_empresa_aerea numero_voo codigo_autorizacao codigo_tipo_linha  \
0                     AZU       4478                  0                 N   
1                     AZU       4479                  0                 N   
2                     AZU       4479                  0                 N   
3                     AZU       4479                  0                 N   
4                     AZU       4479                  0                 N   
...                   ...        ...                ...               ...   
535798                PVV       8653                  9                 X   
535799                PVV       8752                  9                 X   
535800                PVV       8752                  9                 X   
535801                PVV       8752                  2                 I   
535802                PVG       9831                  9                 X   

       icao_aerodromo_origem icao_aerodromo_destino     partida_prevista  \

In [12]:
vra_df.to_csv('./data_export/VRA/vra_data.csv', index=False)

## Carregar dos dados de AIR_CIA
- Normalizar o cabeçalho para snake case
- Separar a coluna 'ICAO IATA' em duas colunas, seu conteúdo está separado por espaço e pode não conter o código IATA, caso não contenha o código IATA, deixe o valor nulo.
- Salvar estes dados

In [13]:
# Diretório onde estão os arquivos CSV
diretorio_air_cia = './data_engineer_test/AIR_CIA/'

In [14]:
# Normalizando as colunas para snake_case
nomes_colunas_air_cia = ['razao_social', 'icao_iata', 'cnpj', 'atividades_aereas',
                         'endereco_sede', 'telefone', 'email', 'decisao_operacional', 
                         'data_decisao_operacional', 'validade_operacional']

In [15]:
# Inicializa um DataFrame vazio para armazenar os dados
df_air_cia = pd.DataFrame()

In [16]:
# Liste todos os arquivos CSV na pasta
for filename in os.listdir(diretorio_air_cia):
    if filename.endswith(".csv"):
        # Crie o caminho completo para o arquivo
        arquivo = os.path.join(diretorio_air_cia, filename)
        
        # Leia o arquivo CSV e adicione os dados ao DataFrame total
        df = pd.read_csv(arquivo, sep=';')
        df_air_cia = pd.concat([df_air_cia, df], ignore_index=True)

In [17]:
# Modificando nome das colunas do dataframe AIR_CIA
df_air_cia.columns = nomes_colunas_air_cia


In [18]:
# separando os valores pelo espaço em icao_iata e colocando nas novas colunas: icao e iata
df_air_cia[['icao', 'iata']] = df_air_cia['icao_iata'].str.split(n=1, expand=True)

In [19]:
# Removendo icao_iata e deixando as novas colunas na mesma posição que ela estava 
posicao_icao_iata = nomes_colunas_air_cia.index('icao_iata')
nomes_colunas_air_cia.pop(posicao_icao_iata)
nomes_colunas_air_cia.insert(posicao_icao_iata, 'icao')
nomes_colunas_air_cia.insert(posicao_icao_iata + 1, 'iata')
df_air_cia = df_air_cia[nomes_colunas_air_cia]

In [20]:
print(df_air_cia.head(5))
print(df_air_cia.shape)

                                        razao_social icao  iata  \
0  AEROSUL TÁXI AÉREO LTDA (EX.: AUSTEN TÁXI AÉRE...  ASO    2S   
1                        ATA - AEROTÁXI ABAETÉ LTDA.  ABJ  None   
2                               OMNI TÁXI AÉREO S.A.  OMI    O1   
3                                PRIME TÁXI AIR LTDA  NaN   NaN   
4                        SIDERAL LINHAS AÉREAS LTDA.  SID    0S   

                 cnpj                                  atividades_aereas  \
0  27.315.694/0001-02  SERVIÇOS AÉREOS PÚBLICOS, TÁXI-AÉREO, TRANSPOR...   
1  14.674.451/0001-19  TÁXI-AÉREO, TRANSPORTE AÉREO NÃO REGULAR, TRAN...   
2  03.670.763/0001-38  AEROFOTOGRAFIA, AEROINSPEÇÃO, SERVIÇOS AÉREOS ...   
3  23.568.370/0001-25  TÁXI-AÉREO, TRANSPORTE AÉREO NÃO REGULAR, SERV...   
4  10.919.908/0001-57  TRANSPORTE AÉREO NÃO REGULAR, TRANSPORTE AÉREO...   

                                       endereco_sede  \
0  RODOVIA PR 218, KM 7, JARDIM UNIVERSITÁRIO, AE...   
1   PÇA. GAGO COUTINHO, S/N,

In [21]:
df_air_cia = df_air_cia.drop_duplicates()

df_air_cia.to_csv('./data_export/AIR_CIA/air_cia_data.csv', index=False)

In [32]:
df_air_cia.columns

Index(['razao_social', 'icao', 'iata', 'cnpj', 'atividades_aereas',
       'endereco_sede', 'telefone', 'email', 'decisao_operacional',
       'data_decisao_operacional', 'validade_operacional'],
      dtype='object')

## Criar nova tabela aerodromos
- Através da API https://rapidapi.com/Active-api/api/airport-info/ trazer os aeródramos através do código ICAO presente nos dados de VRA.
- Salvar estes dados


In [22]:
# Definindo URL e headers para utilizar a API
url = 'https://airport-info.p.rapidapi.com/airport'

query_string = {"icao":"value"}

headers = {
	"X-RapidAPI-Key": "a593951eafmsh253dc4c7ca30e1cp1eec96jsnaef5fd9a9c11",
	"X-RapidAPI-Host": "airport-info.p.rapidapi.com"
}

In [23]:
# Coletando os ICAO únicos
icao_unicos = pd.concat([vra_df['icao_aerodromo_origem'], vra_df['icao_aerodromo_destino']]).unique()

In [24]:
len(icao_unicos)

315

In [27]:
aerodromos_df = pd.DataFrame([])

In [29]:
for icao in icao_unicos:
    query_string['icao'] = icao
#     print(query_string['icao'])
    response = requests.get(url, headers=headers, params=query_string)
    
    if response.status_code == 200:
        if 'error' not in response.json():
            temp_data = pd.DataFrame([response.json()])
            aerodromos_df = pd.concat([aerodromos_df, temp_data], ignore_index=True)
    # print(data)
    # print()

In [31]:
aerodromos_df.columns

Index(['id', 'iata', 'icao', 'name', 'location', 'street_number', 'street',
       'city', 'county', 'state', 'country_iso', 'country', 'postal_code',
       'phone', 'latitude', 'longitude', 'uct', 'website'],
      dtype='object')

## Criar as seguintes views (Priorize o uso de SQL para esta parte):
Para cada companhia aérea trazer a rota mais utilizada com as seguintes informações:
- Razão social da companhia aérea
- Nome Aeroporto de Origem
- ICAO do aeroporto de origem
- Estado/UF do aeroporto de origem
- Nome do Aeroporto de Destino
- ICAO do Aeroporto de destino
- Estado/UF do aeroporto de destino

In [150]:
query = """
        WITH CombinaçõesRankeadas AS (
            SELECT
                vra.icao_empresa_aerea,
                vra.icao_aerodromo_origem,
                vra.icao_aerodromo_destino,
                COUNT(*) AS quantidade,
                ROW_NUMBER() OVER (PARTITION BY vra.icao_empresa_aerea ORDER BY COUNT(*) DESC) AS rank
            FROM vra_df vra
            JOIN df_air_cia f ON vra.icao_empresa_aerea = f.icao
            GROUP BY vra.icao_empresa_aerea, vra.icao_aerodromo_origem, vra.icao_aerodromo_destino
        )
        SELECT
            icao_empresa_aerea,
            icao_aerodromo_origem,
            icao_aerodromo_destino,
            quantidade
        FROM CombinaçõesRankeadas
        WHERE rank = 1;
        """

rotas_mais_utilizadas = sqldf(query)


In [151]:
rotas_mais_utilizadas

,icao_empresa_aerea,icao_aerodromo_origem,icao_aerodromo_destino,quantidade
0,ACN,SWKO,SBEG,34
1,ASO,SBFL,SBCD,26
2,AZU,SBKP,SBRJ,2540
3,GLO,SBRJ,SBSP,2514
4,LTG,SEQM,KMIA,377
5,MWM,SBKP,SBBR,162
6,OMI,SBAR,SBGL,15
7,PAM,SBEG,SBUY,480
8,PTB,SBPS,SBSV,724
9,SID,SBGR,SBRF,752


In [153]:
query = """
        SELECT DISTINCT
            rc.icao_empresa_aerea,
            ac.razao_social AS razao_social_companhia,
            ad_origem.name AS nome_aeroporto_origem,
            rc.icao_aerodromo_origem AS icao_origem,
            ad_origem.county AS estado_origem,
            ad_origem.state AS uf_origem,
            ad_destino.name AS nome_aeroporto_destino,
            rc.icao_aerodromo_destino AS icao_destino,
            ad_destino.county AS estado_destino,
            ad_destino.state AS uf_destino
        FROM rotas_mais_utilizadas rc
        JOIN df_air_cia ac ON rc.icao_empresa_aerea = ac.icao
        JOIN aerodromos_df ad_origem ON rc.icao_aerodromo_origem = ad_origem.icao
        JOIN aerodromos_df ad_destino ON rc.icao_aerodromo_destino = ad_destino.icao;
        """

rotas_mais_utilizadas_info = sqldf(query)

In [155]:
rotas_mais_utilizadas_info

,icao_empresa_aerea,razao_social_companhia,nome_aeroporto_origem,icao_origem,estado_origem,uf_origem,nome_aeroporto_destino,icao_destino,estado_destino,uf_destino
0,ACN,AZUL CONECTA LTDA. (EX TWO TAXI AEREO LTDA),Coari Airport,SWKO,Coari,State of Amazonas,Eduardo Gomes International Airport,SBEG,Manaus,Amazonas
1,ASO,AEROSUL TÁXI AÉREO LTDA (EX.: AUSTEN TÁXI AÉRE...,Hercílio Luz International Airport,SBFL,Florianópolis,Santa Catarina,Caçador Airport,SBCD,Caçador,Santa Catarina
2,AZU,AZUL LINHAS AÉREAS BRASILEIRAS S/A,Viracopos/Campinas International Airport,SBKP,Campinas,São Paulo,Santos Dumont Airport,SBRJ,Rio de Janeiro,Rio de Janeiro
3,GLO,GOL LINHAS AÉREAS S.A. (EX- VRG LINHAS AÉREAS ...,Santos Dumont Airport,SBRJ,Rio de Janeiro,Rio de Janeiro,São Paulo–Congonhas Airport,SBSP,São Paulo,São Paulo
4,LTG,ABSA - AEROLINHAS BRASILEIRAS S.A.,Mariscal Sucre International Airport,SEQM,Distrito Metropolitano de Quito,Pichincha,Miami International Airport,KMIA,Miami-Dade County,Florida
5,MWM,MODERN TRANSPORTE AEREO DE CARGA S.A,Viracopos/Campinas International Airport,SBKP,Campinas,São Paulo,Brasília International Airport (Presidente J. ...,SBBR,Brasília,Distrito Federal
6,OMI,OMNI TÁXI AÉREO S.A.,Santa Maria Airport,SBAR,Santa Barbara County,California,Rio de Janeiro–Galeão International Airport,SBGL,Rio de Janeiro,Rio de Janeiro
7,PTB,PASSAREDO TRANSPORTES AÉREOS S.A.,Porto Seguro Airport,SBPS,Porto Seguro,Bahia,Deputado Luís Eduardo Magalhães International ...,SBSV,Salvador,Bahia
8,SID,SIDERAL LINHAS AÉREAS LTDA.,São Paulo–Guarulhos International Airport,SBGR,Guarulhos,São Paulo,Recife/Guararapes–Gilberto Freyre Internationa...,SBRF,Recife,Pernambuco
9,SUL,ASTA LINHAS AÉREAS LTDA ( EX - AMÉRICA DO SUL ...,Marechal Rondon International Airport,SBCY,Várzea Grande,Mato Grosso,Juína Airport,SWJN,Juína,State of Mato Grosso


## Criar as seguintes views (Priorize o uso de SQL para esta parte):
Para cada aeroporto trazer a companhia aérea com maior atuação no ano com as seguintes informações:
- Nome do Aeroporto
- ICAO do Aeroporto
- Razão social da Companhia Aérea
- Quantidade de Rotas à partir daquele aeroporto
- Quantidade de Rotas com destino àquele aeroporto
- Quantidade total de pousos e decolagens naquele aeroporto

In [ ]:
query = """
WITH AeroportosMaisFrequentes AS (
    SELECT
        a.icao AS ICAO_Aeroporto,
        v.icao_empresa_aerea AS ICAO_Empresa,
        ROW_NUMBER() OVER (PARTITION BY a.icao ORDER BY COUNT(*) DESC) AS rn
    FROM
        aerodromos_df a
    JOIN
        vra_df v ON a.icao = v.icao_aeroporto
    GROUP BY
        a.icao, v.icao_empresa_aerea
)

SELECT
    ICAO_Aeroporto,
    ICAO_Empresa
FROM
    AeroportosMaisFrequentes
WHERE
    rn = 1;
"""

sqldf(query)